## MNIST - Bidirectional Recurrent Neural Network

Inspired from :
    
https://github.com/aymericdamien/TensorFlow-Examples

In [2]:
import tensorflow as tf
import numpy  as np
from tensorflow.examples.tutorials.mnist import input_data

In [3]:
mnist = input_data.read_data_sets("/tmp/data/",one_hot=True)

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [4]:
# No. of training cycles and bacth-size
learning_rate = 0.001
training_epochs = 100000
batch_size = 128
display_step = 100

#Net parameters
n_input = 28
n_steps = 28
n_hidden = 128
n_classes = 10

#Placeholder for feeding the net
x = tf.placeholder("float",[None,n_steps,n_input])
y = tf.placeholder("float",[None,n_classes])

In [5]:
#Weights and biases 
weights = {'out':tf.Variable(tf.random_normal([2*n_hidden,n_classes]))}
biases = {'out':tf.Variable(tf.random_normal([n_classes]))}

Each image is 28 X 28 pixels. We consider each row at a sequence with 28 time-steps. Hence, each image will have 28 sequences with 28 steps.

#### Understanding the input
I am using a batch size of 128. Hence, the input will have dimensions of (128,28,28). The LSTM cell requires a list of tensors. In our case the list will have 28 tensors(equal to number of time-steps) with each tensor of dimension (128 X 28). The function below reshapes the input, and return the model for RNN.

One thing to remember here is that this is bidirectional. Hence, there are forward and backwards memory cells. Thus, number of hidden weights are 2 X n_hidden.

In [6]:
def BiRNN1(x,weights,biases):
    x = tf.transpose(x,[1,0,2])
    x = tf.reshape(x,[-1,n_input])
    x = tf.split(0,n_steps,x)
    with tf.variable_scope('forward'):
        lstm_fw_cell = tf.nn.rnn_cell.BasicLSTMCell(n_hidden,forget_bias=1.0)
        
    with tf.variable_scope('backward'):
        lstm_bw_cell = tf.nn.rnn_cell.BasicLSTMCell(n_hidden,forget_bias=1.0)
    
    
    outputs, _, _ = tf.nn.bidirectional_rnn(lstm_fw_cell,lstm_bw_cell,x,dtype=tf.float32,scope='reuse')
    
    return (tf.matmul(outputs[-1],weights['out']) + biases['out'])

In [7]:
# Creating the RNN
pred = BiRNN1(x,weights,biases)

#Optimization routine
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(pred,y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

#Estimating Accuracy
correct_pred = tf.equal(tf.argmax(pred,1),tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred,tf.float32))

In [8]:
init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    
    step = 1 
    
    while step * batch_size < training_epochs:
        batch_x , batch_y = mnist.train.next_batch(batch_size)
        
        batch_x = batch_x.reshape((batch_size,n_steps,n_input))
        
        sess.run(optimizer,feed_dict = {x:batch_x,y:batch_y})
        
        if step % display_step == 0:
            acc = sess.run(accuracy,feed_dict={x:batch_x,y:batch_y})
            
            loss = sess.run(cost,feed_dict={x:batch_x,y:batch_y})
            
            print ("Iter : " + str(step*batch_size) + 
                  "  Minibatch loss = {:.6f}".format(loss) + 
                  "  Training accuracy = {:.5f}".format(acc))
        
        step += 1
    print ("Optimization Finished !")
    
    test_len = 1024
    test_data = mnist.test.images[:test_len].reshape((-1,n_steps,n_input))
    test_label = mnist.test.labels[:test_len]
    
    print ("Testing accuracy : {}".format(sess.run(accuracy,feed_dict={x:test_data,y:test_label})))
    

Iter : 12800  Minibatch loss = 0.636460  Training accuracy = 0.78125
Iter : 25600  Minibatch loss = 0.375823  Training accuracy = 0.89062
Iter : 38400  Minibatch loss = 0.176354  Training accuracy = 0.93750
Iter : 51200  Minibatch loss = 0.133615  Training accuracy = 0.96094
Iter : 64000  Minibatch loss = 0.079379  Training accuracy = 0.97656
Iter : 76800  Minibatch loss = 0.107371  Training accuracy = 0.96875
Iter : 89600  Minibatch loss = 0.101983  Training accuracy = 0.96875
Optimization Finished !
Testing accuracy : 0.974609375
